<a href="https://colab.research.google.com/github/gpvprasad/Project2022/blob/main/Yolov5CustomTrainBdd100k_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1tEvzlK0inH0guI6IT0NpFmGyeTaR9ohA

Downloading...
From: https://drive.google.com/uc?id=1tEvzlK0inH0guI6IT0NpFmGyeTaR9ohA
To: /content/bdd100k_images_100k.rar
100% 5.73G/5.73G [01:19<00:00, 71.9MB/s]


In [2]:
!unrar x bdd100k_images_100k.rar

Streaming output truncated to the last 5000 lines.
Extracting  bdd100k_images_100k/val/c4756fa2-027007f8.txt                 97%  OK 
Extracting  bdd100k_images_100k/val/c4770a97-1393ce84.jpg                 97%  OK 
Extracting  bdd100k_images_100k/val/c4770a97-1393ce84.txt                 97%  OK 
Extracting  bdd100k_images_100k/val/c4770a97-d8a26f8c.jpg                 97%  OK 
Extracting  bdd100k_images_100k/val/c4770a97-d8a26f8c.txt                 97%  OK 
Extracting  bdd100k_images_100k/val/c4770a97-fb46c2d8.jpg                 97%  OK 
Extracting  bdd100k_images_100k/val/c4770a97-fb46c2d8.txt                 97%  OK 
Extracting  bdd100k_images_100k/val/c477c67b-6eb95776.jpg                 97%  OK 
Extracting  bdd100k_images_100k/val/c477c67b-6eb95776.txt                 97%  OK 
Extracting  bdd100k_images_100k/val/c477c67b-89221134.jpg                 97%  OK 
Extracting  bdd100k_images_10

In [3]:
!rm -rf bdd100k_images_100k.rar 

In [4]:
!mkdir images labels

In [5]:
!mkdir images/train images/val images/test labels/train labels/val labels/test

In [6]:
import os
train_images = [os.path.join('/content/bdd100k_images_100k/train', x) for x in os.listdir('/content/bdd100k_images_100k/train') if x[-3:] == "jpg"]
train_annotations = [os.path.join('/content/bdd100k_images_100k/train', x) for x in os.listdir('/content/bdd100k_images_100k/train') if x[-3:] == "txt"]
val_images = [os.path.join('/content/bdd100k_images_100k/val', x) for x in os.listdir('/content/bdd100k_images_100k/val') if x[-3:] == "jpg"]
val_annotations = [os.path.join('/content/bdd100k_images_100k/val', x) for x in os.listdir('/content/bdd100k_images_100k/val') if x[-3:] == "txt"]
test_images = [os.path.join('/content/bdd100k_images_100k/test', x) for x in os.listdir('/content/bdd100k_images_100k/test') if x[-3:] == "jpg"]

In [7]:
import shutil
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'images/train/')
move_files_to_folder(val_images, 'images/val/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'labels/train/')
move_files_to_folder(val_annotations, 'labels/val/')

In [8]:
!ls

bdd100k_images_100k  images  labels  sample_data


In [9]:
!rm -rf sample_data

In [10]:
!mv images bdd100k_images_100k
!mv labels bdd100k_images_100k

In [11]:
%cd bdd100k_images_100k/

/content/bdd100k_images_100k


In [18]:
import csv
import glob
import os
from tqdm import tqdm
from shutil import copy
import time



# script parameters
original_names = ['car', 'truck', 'bus', 'train', 'person', 'rider', 'traffic sign', 'traffic light', 'motorcycle', 'pedestrian', 'bicycle', 'other vehicle', 'trailer', 'other person']
name_mapping = {'car':'car', 'truck':'truck', 'bus':'bus', 'train':'train', 'person':'person', 'rider':'rider', 'traffic sign':'traffic sign', 'traffic light':'traffic light', 'motorcycle':'motorcycle', 'pedestrian':'person', 'bicycle':'bicycle', 'other vehicle':'truck', 'trailer':'trailer', 'other person':'person'}
dataset_root = '/content/bdd100k_images_100k/'
new_dataset_root = '/content/transformed'

# script code
def create_mapping_from_names(names):
    mapping = dict([(name, str(index)) for index, name in enumerate(names)])
    return mapping

def create_index_mapping(original_names, name_mapping):
    print(f"original names: {original_names}")
    original_mapping = create_mapping_from_names(original_names)
    print(f"original mapping: {original_mapping}")
    print(f"name mapping: {name_mapping}")
    new_names = sorted(list(set([name for name in name_mapping.values()])))
    print(f"new names: {new_names}")
    new_mapping = create_mapping_from_names(new_names)
    print(f"new mapping: {new_mapping}")
    old_index_to_new_index = dict([(original_mapping[old_name], new_mapping[new_name]) for old_name, new_name in name_mapping.items()])
    print(f"old index to new index: {old_index_to_new_index}")
    return new_names, old_index_to_new_index

def convert_label(old_path, new_dir, index_mapping):
    new_lines = []
    with open(old_path, 'r', newline='') as f:
        reader = csv.reader(f, delimiter=' ')
        for label in reader:
            # print(f"old label: {label}")
            old_index = label[0]
            new_index = index_mapping[old_index]
            label[0] = new_index
            # print(f"new label: {label}")
            new_lines.append(' '.join(label))
        
    label_name = old_path.split(os.path.sep)[-1]
    new_path = os.path.join(new_dir, label_name)
    with open(new_path, 'w') as f:
        for index, line in enumerate(new_lines):
            if index != len(new_lines)-1:
                line = line+'\n'
            f.write(line)
        # print(new_lines)
            
def get_labels_in_dir(dir_path):
    return glob.glob(f"{dir_path}/*.txt")

def get_images_in_dir(dir_path):
    return glob.glob(f"{dir_path}/*.jpg")

def gather_data_folders(root):
    folders = []
    if os.path.isdir(os.path.join(root,'labels', 'train')):
        folders.append('train')
    if os.path.isdir(os.path.join(root, 'labels','val')):
        folders.append('val')
    if os.path.isdir(os.path.join(root,'labels', 'test')):
        folders.append('test')
    print(f"data folders: {folders}")
    return folders
    
def convert_folder(root, new_root, folder, index_mapping):

    print(f"converting labels...")
    old_labels_dir = os.path.join(root,'labels', folder)
    new_labels_dir = os.path.join(new_root, folder)
    os.mkdir(new_labels_dir)
    for label in tqdm(get_labels_in_dir(old_labels_dir)):
        convert_label(old_path=label, new_dir=new_labels_dir, index_mapping=index_mapping)
        
    print(f"done converting {folder}")
    
def export_summary(new_root, data_folders, new_names):
    print(f"writing data.yaml")
    with open(os.path.join(new_root, 'data.yaml'), 'w') as f:
        for data_folder in data_folders:
            f.write(f'{data_folder}: ./{data_folder}/images\n')
        f.write('\n')
        f.write(f'nc: {len(new_names)}\n')
        f.write(f'names: {new_names}')

def convert(root, new_root, original_names, name_mapping):
    started = time.time()
    print('1')
    if os.path.isdir(new_root):
        print(f"failure: new root already exists!")
        return
    os.mkdir(new_root)
    print('2')
    data_folders = gather_data_folders(root)
    print('3')
    new_names, index_mapping = create_index_mapping(original_names, name_mapping)
    print(4)
    for folder in data_folders:
        print('5')
        print(f'{folder}')
        convert_folder(root, new_root, folder, index_mapping)

    export_summary(new_root, data_folders, new_names)
    print(f"done converting dataset in {time.time() - started} seconds")
    

convert(root=dataset_root, new_root=new_dataset_root, original_names=original_names, name_mapping=name_mapping)

1
2
data folders: ['train', 'val', 'test']
3
original names: ['car', 'truck', 'bus', 'train', 'person', 'rider', 'traffic sign', 'traffic light', 'motorcycle', 'pedestrian', 'bicycle', 'other vehicle', 'trailer', 'other person']
original mapping: {'car': '0', 'truck': '1', 'bus': '2', 'train': '3', 'person': '4', 'rider': '5', 'traffic sign': '6', 'traffic light': '7', 'motorcycle': '8', 'pedestrian': '9', 'bicycle': '10', 'other vehicle': '11', 'trailer': '12', 'other person': '13'}
name mapping: {'car': 'car', 'truck': 'truck', 'bus': 'bus', 'train': 'train', 'person': 'person', 'rider': 'rider', 'traffic sign': 'traffic sign', 'traffic light': 'traffic light', 'motorcycle': 'motorcycle', 'pedestrian': 'person', 'bicycle': 'bicycle', 'other vehicle': 'truck', 'trailer': 'trailer', 'other person': 'person'}
new names: ['bicycle', 'bus', 'car', 'motorcycle', 'person', 'rider', 'traffic light', 'traffic sign', 'trailer', 'train', 'truck']
new mapping: {'bicycle': '0', 'bus': '1', 'car':

100%|██████████| 69863/69863 [00:37<00:00, 1882.16it/s]


done converting train
5
val
converting labels...


100%|██████████| 10000/10000 [00:01<00:00, 7387.84it/s]


done converting val
5
test
converting labels...


0it [00:00, ?it/s]

done converting test
writing data.yaml
done converting dataset in 38.74738359451294 seconds


In [19]:
!mv /content/transformed ./labels

In [17]:
!rm -rf /content/transformed

In [20]:
%cd ..

/content


In [21]:
!zip -r bdd100k_images_100k.zip bdd100k_images_100k

Streaming output truncated to the last 5000 lines.
  adding: bdd100k_images_100k/images/val/bd98917f-b19d03f7.jpg (deflated 2%)
  adding: bdd100k_images_100k/images/val/c4480d4b-e0efbbca.jpg (deflated 7%)
  adding: bdd100k_images_100k/images/val/b4f65b32-ff644e07.jpg (deflated 3%)
  adding: bdd100k_images_100k/images/val/c900eb3c-4b7befc1.jpg (deflated 1%)
  adding: bdd100k_images_100k/images/val/c56915dc-8d1f7b7a.jpg (deflated 2%)
  adding: bdd100k_images_100k/images/val/bd57e60e-1e11a4e0.jpg (deflated 4%)
  adding: bdd100k_images_100k/images/val/bd656105-db296383.jpg (deflated 1%)
  adding: bdd100k_images_100k/images/val/b77a6ed6-5661f543.jpg (deflated 1%)
  adding: bdd100k_images_100k/images/val/c29d13ba-67447c50.jpg (deflated 2%)
  adding: bdd100k_images_100k/images/val/b35cfd7c-2d625033.jpg (deflated 2%)
  adding: bdd100k_images_100k/images/val/ba99464b-ca1ef680.jpg (deflated 3%)
  adding: bdd100k_images_100k/images/val/b605d5d8-d9310a5c.jpg (deflated 2%)
  adding: bdd100k_images_

In [22]:
!mv bdd100k_images_100k.zip /content/drive/MyDrive/ImageIdetificationProject/Project_work/